In [1]:
import os
import math
import shutil
import warnings
import importlib
import urllib.request as request
from contextlib import closing
from numba import cuda

import cupy
import cudf
import cuml

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

import nvidia.cheminformatics.chembldata as chembldata

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [54]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=5000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    
def plot_kmeans_umap(df2):
    kmeans_float = cuml.KMeans(n_clusters=n_clusters)
    kmeans_float.fit(df2)

    umap = cuml.UMAP(n_neighbors=100,
                     a=1.0,
                     b=1.0,
                     learning_rate=1.0)
    Xt = umap.fit_transform(df2)

    df2['x'] = Xt[0]
    df2['y'] = Xt[1]
    df2['cluster'] = kmeans_float.labels_

    show_cluster_plot(df2)


# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
n_workers = len(client.scheduler_info()['workers'].keys())
client

Client Scheduler: ucx://127.0.0.1:58184 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 3 Cores: 3 Memory: 33.58 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [6]:
%%time
import nvidia.cheminformatics.chembldata
importlib.reload(nvidia.cheminformatics.chembldata)

# cache_directory = '/data/fp'
cache_directory = None
from nvidia.cheminformatics.fingerprint import MorganFingerprint

if cache_directory is None:
    chem_data = chembldata.ChEmblData(db_file=db_file, fp_type=MorganFingerprint)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()
df = dcudf.compute()

CPU times: user 3.09 s, sys: 779 ms, total: 3.87 s
Wall time: 10.4 s


In [7]:
# Compute Tanimoto similarities
fp = cupy.fromDlpack(df.to_dlpack())

# Tanimoto Similarity

In [51]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index#Other_definitions_of_Tanimoto_distance

@cuda.jit
def compute_norms(data, norms):
    i = cuda.grid(1)
    norms[i] = len(data[i])
    for j in range(len(data[i])):
        if data[i][j] != 0:
            value = j + 1
            data[i][j] = value
            norms[i] = norms[i] + (value**2)
    
    if norms[i] != 0:
        norms[i] = math.sqrt(norms[i])

@cuda.jit
def compute_tanimoto_sim_matix(data, norms, sim_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    if i == j:
        sim_array[i][j] = 1
        return
    
    a = data[i]
    b = data[j]
    
    prod = 0
    for k in range(len(data[i])):
        prod = prod + (a[k] * b[k])
        
    a_norm = norms[i]
    b_norm = norms[j]
    
    sim_array[i][j] = (prod / ((a_norm**2 + b_norm**2) - prod))
    
def tanimotoSimilarity(fp):
    norms = cupy.zeros(fp.shape[0])
    compute_norms.forall(norms.shape[0], 1)(fp, norms)

    sim_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_tanimoto_sim_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, norms, sim_array)
    return sim_array

# Jaccard Index Similarity

In [9]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index
@cuda.jit
def compute_jaccard_index_matix(data, index_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    a = data[i]
    b = data[j]
    
    intersects = 0
    unions = 0
    for k in range(len(data[i])):
        if a[k] == 1 and b[k] == 1:
            intersects += 1
            unions += 1
        elif a[k] == 1 or b[k] == 1:
            unions += 1

    index_array[i][j] = (intersects / unions)

def jaccardIndex(fp):
    index_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_jaccard_index_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, index_array)
    return index_array

# Clustering

In [55]:
%%time
from cuml.cluster import DBSCAN

sim_array = tanimotoSimilarity(fp)
# sim_array = jaccardIndex(fp)

# plot_kmeans_umap(df2)

CPU times: user 3.91 s, sys: 27.8 ms, total: 3.93 s
Wall time: 3.88 s


In [11]:
# df2 = cudf.DataFrame.from_gpu_matrix(sim_array)
# plot_kmeans_umap(df2)

In [56]:
df2 = cudf.DataFrame(sim_array)
def plot_kmeans_umap(df2):
    kmeans_float = cuml.KMeans(n_clusters=n_clusters, 
                               random_state=0,
                              init='k-means||')
    kmeans_float.fit(df2)
#     kmeans_float = DBSCAN(eps = 1.0, min_samples = 1)
#     kmeans_float.fit(df2)

    umap = cuml.UMAP(n_neighbors=100,
                     min_dist=0.2,
                     spread=3.0,
                     a=1,
                     b=1,
                     learning_rate=1.0)
    Xt = umap.fit_transform(df2)

    df2['x'] = Xt[0]
    df2['y'] = Xt[1]
    df2['cluster'] = kmeans_float.labels_
    show_cluster_plot(df2)
    
    return  kmeans_float.cluster_centers_

cluster_centers = plot_kmeans_umap(df2)

In [46]:
cluster_centers
# cluster = 1                                         # <- PLEASE CHANGE CLUSTER AS REQUIRED
# query = 'cluster == ' + str(cluster)
# tcdf = df2.query(query)
# show_cluster_plot(tcdf)


,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.187303,0.176938,0.142790,0.175590,0.191129,0.207729,0.173890,0.184287,0.181821,0.088165,...,0.104886,0.183477,0.162084,0.140074,0.105502,0.145665,0.218319,0.232137,0.136444,0.147817
1,0.122404,0.115570,0.124017,0.118606,0.126182,0.121712,0.116737,0.124158,0.132449,0.114185,...,0.110358,0.118519,0.119550,0.085285,0.103469,0.092430,0.090553,0.120538,0.093968,0.090104
2,0.104698,0.108378,0.099625,0.103057,0.108683,0.109314,0.103993,0.108086,0.108379,0.092125,...,0.074081,0.082177,0.119070,0.136906,0.068248,0.136546,0.083838,0.111492,0.135062,0.142462
3,0.157885,0.155256,0.152132,0.152809,0.162485,0.161626,0.148352,0.159583,0.163885,0.109719,...,0.132913,0.135291,0.166586,0.188509,0.117901,0.185426,0.135488,0.157627,0.200559,0.201856
4,0.167313,0.171578,0.177181,0.175265,0.176812,0.179351,0.158834,0.170172,0.169373,0.092606,...,0.139974,0.119623,0.144000,0.093809,0.137846,0.113233,0.117997,0.167395,0.114301,0.100427
5,0.059758,0.062360,0.051527,0.067237,0.065911,0.068247,0.056682,0.060485,0.062129,0.051989,...,0.047999,0.057716,0.083892,0.066862,0.047787,0.074063,0.061751,0.063806,0.070423,0.067505
6,0.135436,0.136793,0.116835,0.136542,0.147631,0.161865,0.128217,0.138055,0.142976,0.087697,...,0.129128,0.152588,0.161585,0.100878,0.105424,0.111930,0.143542,0.133076,0.115116,0.103960
7,0.149979,0.142118,0.144204,0.142238,0.150560,0.149020,0.139918,0.145839,0.158863,0.118465,...,0.114783,0.118447,0.142163,0.149879,0.111905,0.153437,0.111172,0.142790,0.152430,0.159890
8,0.215388,0.208034,0.182199,0.211929,0.229911,0.245091,0.210059,0.221699,0.223168,0.108284,...,0.126598,0.198242,0.162069,0.099267,0.124088,0.105101,0.148813,0.210842,0.099652,0.107964
9,0.140452,0.139226,0.117194,0.137176,0.145337,0.163792,0.131101,0.140486,0.145394,0.104096,...,0.086656,0.122798,0.152809,0.232893,0.079874,0.232553,0.136942,0.152077,0.231951,0.234716
